In [55]:
import sys, pickle, json, os, re
import numpy as np
from collections import defaultdict
from pprint import pprint
from typing import List, Dict, Any
sys.path.append('../src/')
sys.path.append("../../scripts/")
from utils import find_repo_root

import matplotlib.pyplot as plt
from plt_config import use_latex, TU_GREY, TU_RED
use_latex()
project_root_dir = find_repo_root()
print("Project root directory:", project_root_dir)

Project root directory: /home/etschgi1/REPOS/Masterarbeit


In [56]:
ray_folder = "/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_full_testing/tune_logs/MGNN_hyp_small_full_full_mat_loss_server.py"
trials = []
for folder in os.listdir(ray_folder):
    sub_folder_path = os.path.join(ray_folder, folder)
    if os.path.exists(os.path.join(sub_folder_path, "eval_res_reeval.json")):
        with open(os.path.join(sub_folder_path, "eval_res_reeval.json")) as f:
            eval_res = json.load(f)
        with open(os.path.join(sub_folder_path, "eval_res.json")) as f:
            eval_res_old = json.load(f)
        eval_res["cummulative_stats"]["iterations"] = eval_res_old["cummulative_stats"]["iterations"]
        with open(os.path.join(sub_folder_path, "params.json")) as f:
            params = json.load(f)
        with open(os.path.join(sub_folder_path, "result.json")) as f:
            last_text = f.readlines()[-1]
            epochs_trained = re.search('epoch": [0-9]*', last_text).group(0).split(": ")[-1]
        params["folder"] = folder
        params["epochs_trained"] = int(epochs_trained)
        trials.append((params, eval_res))
len(trials)

12

In [57]:
trials.sort(key=lambda x:x[1]["cummulative_stats"]["iterations"]["mean"])
print(f"Found {len(trials)} trials in {ray_folder}")
for trial in trials: 
    print(trial[0]["folder"])
    # print(trial[1]["cummulative_stats"])
    # print(trial[0]["epochs_trained"])
    # print("---")


Found 12 trials in /home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_full_testing/tune_logs/MGNN_hyp_small_full_full_mat_loss_server.py
hyperopt_train_036e6_00043_43_batch_size=8,data_aug_factor=4,edge_threshold_val=3.4020,hidden_dim=128,lr=0.0007,message_net_dropou_2025-07-18_12-18-58
hyperopt_train_036e6_00078_78_batch_size=8,data_aug_factor=1,edge_threshold_val=2.5786,hidden_dim=256,lr=0.0009,message_net_dropou_2025-07-18_15-39-03
hyperopt_train_036e6_00042_42_batch_size=8,data_aug_factor=1,edge_threshold_val=3.9776,hidden_dim=256,lr=0.0016,message_net_dropou_2025-07-18_12-14-41
hyperopt_train_036e6_00032_32_batch_size=8,data_aug_factor=1,edge_threshold_val=3.2088,hidden_dim=256,lr=0.0003,message_net_dropou_2025-07-18_11-39-56
hyperopt_train_036e6_00027_27_batch_size=8,data_aug_factor=2,edge_threshold_val=2.8523,hidden_dim=128,lr=0.0008,message_net_dropou_2025-07-18_11-16-53
hyperopt_train_036e6_00033_33_batch_size=8,data_aug_factor=1,edge_threshold_val=3.2851,hidden_

In [60]:
top = 10
table_data = []
for i, (params, metrics) in enumerate(trials[:top]):
    metrics = metrics["cummulative_stats"]
    row = {
        'ID': i,
        'Iterations': f"{float(metrics['iterations']['mean']):.1f}({float(metrics['iterations']['std']):.1f})",
        r"$\Delta E_\text{HF}$": f"{float(metrics['energy_abs']['mean']):.1f}({float(metrics['energy_abs']['std']):.1f})",
        r"$\delta E_\text{HF}$": f"{float(metrics['energy_rel']['mean']):.4f}({float(metrics['energy_rel']['std']):.4f})",
        'DIIS error': f"{float(metrics['diis']['mean']):.3f}({float(metrics['diis']['std']):.3f})",
        r"$\overline{\text{RMSE}}$": f"{float(metrics['rmse']['mean']):.4f}({float(metrics['rmse']['std']):.4f})"
    }
    table_data.append(row)

import pandas as pd
df = pd.DataFrame(table_data)

latex = df.to_latex(index=False)
print(latex)

\begin{tabular}{rlllll}
\toprule
ID & Iterations & $\Delta E_\text{HF}$ & $\delta E_\text{HF}$ & DIIS error & $\overline{\text{RMSE}}$ \\
\midrule
0 & 11.9(2.1) & 1.2(5.5) & 0.0009(0.0031) & 0.034(0.011) & 0.0088(0.0026) \\
1 & 12.0(1.7) & 0.9(1.7) & 0.0005(0.0011) & 0.034(0.013) & 0.0092(0.0024) \\
2 & 12.0(1.8) & -1.3(4.2) & -0.0009(0.0024) & 0.035(0.011) & 0.0093(0.0025) \\
3 & 12.0(1.9) & -1.6(2.7) & -0.0010(0.0017) & 0.034(0.013) & 0.0090(0.0026) \\
4 & 12.0(1.8) & -0.2(2.1) & -0.0002(0.0017) & 0.044(0.048) & 0.0090(0.0032) \\
5 & 12.0(1.7) & 0.3(2.7) & 0.0001(0.0022) & 0.033(0.013) & 0.0090(0.0025) \\
6 & 12.0(2.0) & 0.3(1.9) & 0.0002(0.0013) & 0.033(0.013) & 0.0086(0.0028) \\
7 & 12.1(1.6) & -0.7(2.5) & -0.0007(0.0019) & 0.038(0.018) & 0.0091(0.0029) \\
8 & 12.1(2.1) & 0.1(1.9) & -0.0000(0.0013) & 0.035(0.012) & 0.0093(0.0027) \\
9 & 12.1(1.8) & 1.3(2.3) & 0.0008(0.0013) & 0.036(0.014) & 0.0090(0.0024) \\
\bottomrule
\end{tabular}



In [59]:
table_data = []
for i, (params, metrics) in enumerate(trials[:top]):
    metrics = metrics["cummulative_stats"]
    row = {
        'Parameter': int(i),
        'Hidden Dimension': int(params["hidden_dim"]),
        'Batch Size': int(params["batch_size"]),
        "Data aug.": f'{float(params["data_aug_factor"]):.2f}',
        "Edge thres.": f'{float(params["edge_threshold_val"]):.2f}',
        "Message p. steps": int(params["message_passing_steps"]),
        "Message Net Dropout": f'{float(params["message_net_dropout"]):.2f}',
        "Message Net Layers": int(params["message_net_layers"]),
        "Learn Rate": f'{float(params["lr"]):.2e}',
        "Weight Decay": f'{float(params["weight_decay"]):.2e}',
    }
    table_data.append(row)

import pandas as pd
df = pd.DataFrame(table_data).T


latex = df.to_latex(index=True)
print(latex)

\begin{tabular}{lllllllllll}
\toprule
 & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\midrule
Parameter & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
Hidden Dimension & 128 & 256 & 256 & 256 & 128 & 512 & 512 & 512 & 512 & 128 \\
Batch Size & 8 & 8 & 8 & 8 & 8 & 8 & 8 & 16 & 8 & 8 \\
Data aug. & 4.00 & 1.00 & 1.00 & 1.00 & 2.00 & 1.00 & 4.00 & 3.00 & 1.00 & 2.00 \\
Edge thres. & 3.40 & 2.58 & 3.98 & 3.21 & 2.85 & 3.29 & 3.27 & 2.72 & 3.44 & 2.30 \\
Message p. steps & 4 & 5 & 3 & 2 & 2 & 4 & 3 & 3 & 2 & 4 \\
Message Net Dropout & 0.09 & 0.16 & 0.25 & 0.21 & 0.11 & 0.04 & 0.25 & 0.04 & 0.18 & 0.19 \\
Message Net Layers & 3 & 3 & 3 & 3 & 4 & 3 & 3 & 2 & 3 & 2 \\
Learn Rate & 6.90e-04 & 9.31e-04 & 1.61e-03 & 2.75e-04 & 8.03e-04 & 1.51e-04 & 1.26e-04 & 5.85e-04 & 1.09e-03 & 6.50e-04 \\
Weight Decay & 7.15e-05 & 5.46e-05 & 3.58e-05 & 4.69e-05 & 6.06e-04 & 3.99e-04 & 2.56e-04 & 1.45e-04 & 1.84e-05 & 1.74e-05 \\
\bottomrule
\end{tabular}

